# ISSM helper

A collection of functions to help run ISSM a little more intuitively. 

In [ ]:
from project3d import project3d
from project2d import project2d
import numpy as np
from solve import solve
import os, sys
from setflowequation import setflowequation

## ```solve_quiet```

Avoid all ISSM's default output, which gets annoying if iterating over multiple timesteps. 

In [ ]:
class Null:
    
    def write(self, *_): pass
    def flush(self): pass

null = Null()

def solve_quiet(md, kind):
    
    dn = os.open(os.devnull, os.O_WRONLY)
    o1, o2 = os.dup(1), os.dup(2)
    s1, s2 = sys.stdout, sys.stderr
    sys.stdout = null; sys.stderr = null
    os.dup2(dn, 1); os.dup2(dn, 2); os.close(dn)
    
    try:
        return solve(md, kind)
    finally:
        os.dup2(o1, 1); os.dup2(o2, 2)
        os.close(o1); os.close(o2)
        sys.stdout = s1; sys.stderr = s2

## ```initialize_model```

Set up model parameters that are unlikely to change during a simulation. 

In [ ]:
def initialize_model(md, **kwargs):
    
    T = kwargs.get('temperature', 273.15)
    friction_law = kwargs.get('friction_law', lambda x: x)
    p = kwargs.get('p', 1)
    q = kwargs.get('q', 1)
    Γ_x = kwargs.get('dirichlet_x', np.zeros(md.mesh.numberofvertices).astype(bool))
    Γ_y = kwargs.get('dirichlet_y', np.zeros(md.mesh.numberofvertices).astype(bool))
    Γ_in = kwargs.get('dirichlet_H', np.zeros(md.mesh.numberofvertices).astype(bool))
    name = kwargs.get('name', 'name')
    vertical_layers = kwargs.get('vertical_layers', 5)
    extrusion_exponent = kwargs.get('extrusion_exponent', 1)
    approx = kwargs['approximation']

    num_e = md.mesh.numberofelements
    num_v = md.mesh.numberofvertices

    ####################
    ## approximation ###
    ####################

    md = setflowequation(md, approx, 'all')

    ################
    ### rheology ###
    ################
    
    T_t = 263.15 #transition temperature (K)
    Q = 6e4 if T < T_t else 115e3 #activation enegy (J/mol), depends on temperature
    A_0 = 3.5e-25 #rate prefactor (s^-1 Pa^-3)
    R = 8.314 #gas constant (J mol^-1 K^-1)
    A_val = A_0*np.exp(-Q/R*(1/T - 1/T_t)) #final rate factor (s^-1 Pa^-3)
    B_val = A_val**(-1/md.materials.rheology_n)
    md.materials.rheology_B = B_val*np.ones(num_v)
    md.miscellaneous.rheology_A = A_val

    ################
    ### friction ###
    ################

    md.friction.p = p*np.ones(num_e) if np.isscalar(p) else p
    md.friction.q = q*np.ones(num_e) if np.isscalar(q) else q

    ################################
    ### set dirichlet boundaries ###
    ################################

    md.stressbalance.spcvx = np.nan*np.ones(num_v)
    md.stressbalance.spcvy = np.nan*np.ones(num_v)
    md.stressbalance.spcvz = np.nan*np.ones(num_v)
    md.masstransport.spcthickness = np.nan*np.ones(num_v)

    Γ_x = np.isfinite(np.array(Γ_x)) if np.nan in Γ_x else np.array(Γ_x).astype(bool)
    Γ_y = np.isfinite(np.array(Γ_y)) if np.nan in Γ_y else np.array(Γ_y).astype(bool)
    Γ_in = np.isfinite(np.array(Γ_in)) if np.nan in Γ_in else np.array(Γ_in).astype(bool)

    try: 
        md.stressbalance.spcvx[Γ_x] = 1
        md.stressbalance.spcvy[Γ_y] = 1
        md.masstransport.spcthickness[Γ_in] = 1
    except:
        proj = lambda x: project3d(md, 'vector', x, 'type', 'node')
        md.stressbalance.spcvx[proj(Γ_x)] = 1
        md.stressbalance.spcvy[proj(Γ_y)] = 1
        md.masstransport.spcthickness[proj(Γ_in)] = 1     
        
    ##############
    ### extras ###
    ##############

    md.stressbalance.referential = np.nan*np.ones((num_v, 6))
    md.stressbalance.loadingforce = np.zeros((num_v, 3))
    md.basalforcings.groundedice_melting_rate = np.zeros(num_v)
    md.basalforcings.floatingice_melting_rate = np.zeros(num_v)
    md.geometry.thickness = np.ones(num_v)
    md.geometry.surface = np.ones(num_v)
    md.geometry.base = np.zeros(num_v)
    md.smb.initialize(md) #initialize an empty SMB field

    ###############
    ### storage ###
    ###############

    md.miscellaneous.temperature = T
    md.miscellaneous.name = name
    md.miscellaneous.friction_p = p
    md.miscellaneous.friction_q = q
    md.miscellaneous.vertical_layers = vertical_layers if md.flowequation.isHO else np.nan
    md.miscellaneous.extrusion_exponent = extrusion_exponent if md.flowequation.isHO else np.nan
    md.miscellaneous.friction_law = friction_law

    ############################
    ### extrude if necessary ###
    ############################

    if not hasattr(md.mesh, 'numberofvertices2d'):
        md.miscellaneous.mesh2d = md.mesh
        if md.flowequation.isHO:
            md = md.extrude(vertical_layers, extrusion_exponent, 1)
            md.miscellaneous.zeta = md.mesh.z #this will tell us how z is distributed proportionally

## ```diagnostic_solve```

Use mirrors ```icepack```'s function of the same name. 

In [ ]:
def diagnostic_solve(md, **kwargs):
    
    H = kwargs['thickness']
    s = kwargs.get('surface', None)
    b = kwargs.get('base', None)
    u = kwargs.get('velocity', None)
    C = kwargs.get('friction', None)
    quiet = kwargs.get('quiet', True)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    ######################
    ### basal friction ###
    ######################

    if C is None:
        if md.flowequation.isSSA:
            C = 0
        else:
            raise RuntimeError('Must specify friction coefficient.')
    
    friction_law = md.miscellaneous.friction_law
    C = friction_law(C)
    md.friction.coefficient = C*np.ones(md.mesh.numberofvertices) if np.isscalar(C) else C

    ####################
    ### set geometry ###
    ####################

    H[H <= 0] = 1e-10
    if s is None:
        if md.flowequation.isSSA:
            s = (1 - ϱ)*H
        else:
            raise RuntimeError('Must specify surface elevation profile.')
    if b is None:
        b = s - H

    if np.size(H) == md.miscellaneous.mesh2d.numberofvertices and md.flowequation.isHO:
        proj = lambda x: project3d(md, 'vector', x, 'type', 'node')
        H, s, b = proj(H), proj(s), proj(b)

    md.geometry.thickness = H
    md.geometry.surface = s
    md.geometry.base = b

    ####################################
    ### velocity boundary conditions ###
    ####################################

    Γ_x = np.isfinite(md.stressbalance.spcvx)
    Γ_y = np.isfinite(md.stressbalance.spcvy)

    if True in Γ_x | Γ_y:
        if u is None:
            raise RuntimeError('Must provide initial velocity to enforce Dirichlet BCs.')
        else: 
            md.stressbalance.spcvx[Γ_x] = u[0] if np.isscalar(u[0]) else u[0][Γ_x]
            md.stressbalance.spcvy[Γ_y] = u[1] if np.isscalar(u[1]) else u[1][Γ_y]

    #####################
    ### update mesh.z ###
    #####################

    if md.flowequation.isHO:
        ζ = md.miscellaneous.zeta
        md.mesh.z = b + ζ*H

    ##################
    ### solve step ###
    ##################

    md = solve_quiet(md, 'Stressbalance') if quiet else solve(md, 'Stressbalance')
    solution = md.results.StressbalanceSolution
    u_x, u_y = solution.Vx.flatten(), solution.Vy.flatten()
    u = (u_x, u_y)
    if md.flowequation.isHO:
        u = (u_x, u_y, solution.Vz.flatten())
    return u          

## ```prognostic_solve```

Use mirrors ```icepack```'s function of the same name, but returns both thickness and surface elevation fields. 

In [ ]:
def prognostic_solve(md, Δt, **kwargs):
    
    H = kwargs['thickness']
    H_in = kwargs.get('thickness_inflow', None)
    s = kwargs.get('surface', None)
    b = kwargs.get('base', None)
    u = kwargs['velocity']
    quiet = kwargs.get('quiet', True)
    stab = kwargs.get('stabilization', 0)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    ####################
    ### set geometry ###
    ####################

    H[H <= 0] = 1e-10
    if s is None:
        if md.flowequation.isSSA:
            s = (1 - ϱ)*H
        else:
            raise RuntimeError('Must specify surface elevation profile.')
    if b is None:
        b = s - H

    if np.size(H) == md.miscellaneous.mesh2d.numberofvertices and md.flowequation.isHO:
        proj = lambda x: project3d(md, 'vector', x, 'type', 'node')
        H, s, b = proj(H), proj(s), proj(b)
        H_in = H_in if np.isscalar(H_in) else proj(H_in)

    md.geometry.thickness = H
    md.geometry.surface = s
    md.geometry.base = b

    #####################
    ### update mesh.z ###
    #####################

    if md.flowequation.isHO:
        ζ = md.miscellaneous.zeta
        md.mesh.z = b + ζ*H

    ##################################
    ### inflow boundary conditions ###
    ##################################

    Γ_in = np.isfinite(md.masstransport.spcthickness)
    if True in Γ_in:
        if H_in is None:
            raise RuntimeError('Must provide thickness constraint to enforce Dirichlet BC.')
        else:
            md.masstransport.spcthickness[Γ_in] = H_in if np.isscalar(H_in) else H_in[Γ_in]

    ############################
    ### transient patameters ###
    ############################

    md.timestepping.time_step = Δt #try to model only a single mass-transport step
    md.timestepping.final_time = Δt
    md.transient.isstressbalance = 0 #solve the stress balance?
    md.transient.ismasstransport = 1 #solve mass transport?
    md.transient.isthermal = 0 #don't bother with heat transport
    md.initialization.vx = u[0]
    md.initialization.vy = u[1]
    if stab is not None:
        md.masstransport.stabilization = stab

    ##################
    ### solve step ###
    ##################

    md = solve_quiet(md, 'Transient') if quiet else solve(md, 'Transient')
    solution = md.results.TransientSolution[-1]
    H_new = solution.Thickness.flatten()
    s_new = b + H_new
    if np.size(H) > md.miscellaneous.mesh2d.numberofvertices:
        proj = lambda x: project2d(md, x, 1)
        H_new, s_new = proj(H_new), proj(s_new)
    return H_new, s_new

## ```coupled_solve```

Performs both diagnostic and prognostic solves in a single step. Somewhat faster that splitting. Output is the pair ```(u, H, s)```, where the dimension of ```u``` depends on the approximation.  

In [ ]:
def coupled_solve(md, Δt, **kwargs):
    
    H = kwargs['thickness']
    H_in = kwargs.get('thickness_inflow', None)
    s = kwargs.get('surface', None)
    b = kwargs.get('base', None)
    u = kwargs['velocity']
    C = kwargs.get('friction', None)
    quiet = kwargs.get('quiet', True)
    stab = kwargs.get('stabilization', 0)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    ######################
    ### basal friction ###
    ######################

    if C is None:
        if md.flowequation.isSSA:
            C = 0
        else:
            raise RuntimeError('Must specify friction coefficient.')
    
    friction_law = md.miscellaneous.friction_law
    C = friction_law(C)
    md.friction.coefficient = C*np.ones(md.mesh.numberofvertices) if np.isscalar(C) else C

    ####################
    ### set geometry ###
    ####################

    H[H <= 0] = 1e-10
    if s is None:
        if md.flowequation.isSSA:
            s = (1 - ϱ)*H
        else:
            raise RuntimeError('Must specify surface elevation profile.')
    if b is None:
        b = s - H
    
    if np.size(H) == md.miscellaneous.mesh2d.numberofvertices and md.flowequation.isHO:
        proj = lambda x: project3d(md, 'vector', x, 'type', 'node')
        H, s, b = proj(H), proj(s), proj(b)
        H_in = H_in if np.isscalar(H_in) else proj(H_in)

    md.geometry.thickness = H
    md.geometry.surface = s
    md.geometry.base = b

    #####################
    ### update mesh.z ###
    #####################

    if md.flowequation.isHO:
        ζ = md.miscellaneous.zeta
        md.mesh.z = b + ζ*H

    ###########################
    ### boundary conditions ###
    ###########################

    Γ_x = np.isfinite(md.stressbalance.spcvx)
    Γ_y = np.isfinite(md.stressbalance.spcvy)
    Γ_in = np.isfinite(md.masstransport.spcthickness)

    if True in Γ_x | Γ_y:
        if u is None:
            raise RuntimeError('Must provide initial velocity to enforce Dirichlet BCs.')
        else: 
            md.stressbalance.spcvx[Γ_x] = u[0] if np.isscalar(u[0]) else u[0][Γ_x]
            md.stressbalance.spcvy[Γ_y] = u[1] if np.isscalar(u[1]) else u[1][Γ_y]

    if True in Γ_in:
        if H_in is None:
            raise RuntimeError('Must provide thickness constraint to enforce Dirichlet BC.')
        else:
            md.masstransport.spcthickness[Γ_in] = H_in if np.isscalar(H_in) else H_in[Γ_in]

    ############################
    ### transient patameters ###
    ############################

    md.timestepping.time_step = Δt #try to model only a single mass-transport step
    md.timestepping.final_time = Δt
    md.transient.isstressbalance = 1 #solve the stress balance?
    md.transient.ismasstransport = 1 #solve mass transport?
    md.transient.isthermal = 0 #don't bother with heat transport
    md.initialization.vx = u[0]
    md.initialization.vy = u[1]
    if stab is not None:
        md.masstransport.stabilization = stab

    ##################
    ### solve step ###
    ##################

    md = solve_quiet(md, 'Transient') if quiet else solve(md, 'Transient')
    solution = md.results.TransientSolution[-1]
    u_x, u_y = solution.Vx.flatten(), solution.Vy.flatten()
    u = (u_x, u_y)
    if md.flowequation.isHO:
        u = (u_x, u_y, solution.Vz.flatten())
    H_new = solution.Thickness.flatten()
    s_new = b + H_new
    if np.size(H) > md.miscellaneous.mesh2d.numberofvertices:
        proj = lambda x: project2d(md, x, 1)
        H_new, s_new = proj(H_new), proj(s_new)
    return u, H_new, s_new

## Convert to .py script

In [ ]:
# !jupyter nbconvert --to script issm_helper.ipynb